# Reading the necessary libs and setting the stage

In [1]:
# creation of data sources using 
import boto3
import json

client  = boto3.client('quicksight')
quicksight  = boto3.client('quicksight')
account_id = '854807003520'
ds_id = '63aa357c-fd33-4ebe-b071-312bc3803c5b'

# ________________________________list data source______________________________________________
list_datasource = quicksight.list_data_sources(AwsAccountId= account_id)
list_datasource

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


AccessDeniedException: An error occurred (AccessDeniedException) when calling the ListDataSources operation: User: arn:aws:sts::014257795134:assumed-role/ds-mlops-sagemaker-role/SageMaker is not authorized to access this resource

# Describing the .JSON for QuickSight

In [9]:
# _________________________________describe data source_________________________________________
describe_data_source = quicksight.describe_data_source(AwsAccountId=account_id,
                                                       DataSourceId=ds_id)
describe_data_source

{'ResponseMetadata': {'RequestId': '4c3e6e33-5670-426c-ac36-53bb1cdabd39',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:22:16 GMT',
   'content-type': 'application/json',
   'content-length': '1179',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4c3e6e33-5670-426c-ac36-53bb1cdabd39'},
  'RetryAttempts': 0},
 'Status': 200,
 'DataSource': {'Arn': 'arn:aws:quicksight:us-east-1:854807003520:datasource/63aa357c-fd33-4ebe-b071-312bc3803c5b',
  'DataSourceId': '63aa357c-fd33-4ebe-b071-312bc3803c5b',
  'Name': 'dsmlops_datadrift_analysis',
  'Type': 'ATHENA',
  'Status': 'CREATION_SUCCESSFUL',
  'CreatedTime': datetime.datetime(2022, 5, 26, 14, 53, 24, 666000, tzinfo=tzlocal()),
  'LastUpdatedTime': datetime.datetime(2022, 5, 26, 14, 53, 24, 666000, tzinfo=tzlocal()),
  'DataSourceParameters': {'AthenaParameters': {'WorkGroup': 'primary'}},
  'SslProperties': {'DisableSsl': False}},
 'RequestId': '4c3e6e33-5670-426c-ac36-53bb1cdabd39'}

In [10]:

with open ("data_drift_source.json", "w") as outfile:
    json.dump(describe_data_source, outfile, indent=4, sort_keys=True, default=str) # dictionary to json

# Describing the Dataset For QuickSight

In [11]:
#_____________________________________ Extracting variables from Json______________________________________

ds_Id = describe_data_source['DataSource']['DataSourceId']
ds_Name = describe_data_source['DataSource']['Name']
ds_parameters = describe_data_source['DataSource']['DataSourceParameters']
ssl = describe_data_source['DataSource']['SslProperties']
type_ds = describe_data_source['DataSource']['Type']

In [12]:
#________________________ Function to create data sources_______________________________________________________
def create_ds(client, account_id,Data_source_Id,data_source_name,DataSourceParameter,SslProperty):
    DS_creation = client.create_data_source(
    AwsAccountId=account_id,
    DataSourceId=Data_source_Id,      #Ds_id, #we'll provide the ds id
    Name=data_source_name, #display name for the data source
    Type='ATHENA',
    DataSourceParameters=DataSourceParameter,
#     Credentials=Credential,
#     Permissions=[
#         {
#             'Principal': arn,
#             'Actions': [ "athena:*", "quicksight:*", "quicksight:CreateDataSource"]
#         },
#     ],
#     VpcConnectionProperties={
#         'VpcConnectionArn': 'string'
#     },
    SslProperties=SslProperty)
    return DS_creation



In [23]:
# _____________________________________List dataset _______________________________________________________________
list_dataset = quicksight.list_data_sets(AwsAccountId=account_id)

list_dataset


{'ResponseMetadata': {'RequestId': '1aec8436-bf0b-407b-ae01-ecea31a94514',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:36:39 GMT',
   'content-type': 'application/json',
   'content-length': '12086',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1aec8436-bf0b-407b-ae01-ecea31a94514'},
  'RetryAttempts': 0},
 'Status': 200,
 'DataSetSummaries': [{'Arn': 'arn:aws:quicksight:us-east-1:854807003520:dataset/03c1a407-8e03-49cf-b740-65e99febe0a9',
   'DataSetId': '03c1a407-8e03-49cf-b740-65e99febe0a9',
   'Name': 'scoringtable',
   'CreatedTime': datetime.datetime(2022, 5, 24, 13, 35, 37, 779000, tzinfo=tzlocal()),
   'LastUpdatedTime': datetime.datetime(2022, 5, 24, 13, 35, 37, 779000, tzinfo=tzlocal()),
   'ImportMode': 'DIRECT_QUERY',
   'RowLevelPermissionTagConfigurationApplied': False,
   'ColumnLevelPermissionRulesApplied': False},
  {'Arn': 'arn:aws:quicksight:us-east-1:854807003520:dataset/0f8cb37d-f715-44d4-8be4-1a144d6e238f',
   'DataSetId': '0f

In [14]:
for i in list_dataset['DataSetSummaries']:
    if i ['Name'] == 'Drift_Dataset_MLOPs':
        print(i)

In [24]:
DataSetId = '54fb1fd7-9984-4400-a692-27e761375505'

# _________________________________Describe dataset ________________________________________________________________
describe_dataset = client.describe_data_set(AwsAccountId=account_id, DataSetId= DataSetId)

describe_dataset

{'ResponseMetadata': {'RequestId': '8ae46cf3-927f-4b68-9afa-855fb1817fd6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:37:41 GMT',
   'content-type': 'application/json',
   'content-length': '5919',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8ae46cf3-927f-4b68-9afa-855fb1817fd6'},
  'RetryAttempts': 0},
 'Status': 200,
 'DataSet': {'Arn': 'arn:aws:quicksight:us-east-1:854807003520:dataset/54fb1fd7-9984-4400-a692-27e761375505',
  'DataSetId': '54fb1fd7-9984-4400-a692-27e761375505',
  'Name': 'datadrift',
  'CreatedTime': datetime.datetime(2022, 5, 26, 14, 58, 43, 80000, tzinfo=tzlocal()),
  'LastUpdatedTime': datetime.datetime(2022, 6, 1, 18, 12, 47, 63000, tzinfo=tzlocal()),
  'PhysicalTableMap': {'7ba3a605-150b-4617-b523-cefcbcb33de3': {'RelationalTable': {'DataSourceArn': 'arn:aws:quicksight:us-east-1:854807003520:datasource/63aa357c-fd33-4ebe-b071-312bc3803c5b',
     'Catalog': 'AwsDataCatalog',
     'Schema': 'modelmonitordb',
     'Name': 'd

In [25]:
with open ("drift_data_set.json", "w") as outfile:
    json.dump(describe_dataset, outfile, indent=4, sort_keys=True, default=str) # dictionary to json

# ---------------------------- List analysis ---------------------------------



In [26]:
list_analysis = client.list_analyses(AwsAccountId=account_id)
list_analysis

{'ResponseMetadata': {'RequestId': '4e670ad2-74e8-4bbe-8335-b805d3a2c5cb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:37:56 GMT',
   'content-type': 'application/json',
   'content-length': '9918',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4e670ad2-74e8-4bbe-8335-b805d3a2c5cb'},
  'RetryAttempts': 0},
 'Status': 200,
 'AnalysisSummaryList': [{'Arn': 'arn:aws:quicksight:us-east-1:854807003520:analysis/087a00f6-39f5-4190-a807-13591e97070a',
   'AnalysisId': '087a00f6-39f5-4190-a807-13591e97070a',
   'Name': 'Business Review analysis',
   'Status': 'CREATION_SUCCESSFUL',
   'CreatedTime': datetime.datetime(2022, 6, 1, 19, 19, 45, 94000, tzinfo=tzlocal()),
   'LastUpdatedTime': datetime.datetime(2022, 6, 1, 19, 19, 45, 94000, tzinfo=tzlocal())},
  {'Arn': 'arn:aws:quicksight:us-east-1:854807003520:analysis/16255761-f6bb-4b46-9f79-5ab75b68da11',
   'AnalysisId': '16255761-f6bb-4b46-9f79-5ab75b68da11',
   'Name': 'Web and Social Media Analytics analy

In [28]:
analysis_id = '5aeb6958-4e0c-4fe0-b720-a4d9750612f9'

### --------------------- Desribe analysis -------------------------------------------------------
describe_analysis = client.describe_analysis(AwsAccountId=account_id, AnalysisId=analysis_id)
describe_analysis

{'ResponseMetadata': {'RequestId': '47da5738-9cf9-44a9-9d97-c65532219c9f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:39:20 GMT',
   'content-type': 'application/json',
   'content-length': '989',
   'connection': 'keep-alive',
   'x-amzn-requestid': '47da5738-9cf9-44a9-9d97-c65532219c9f'},
  'RetryAttempts': 0},
 'Status': 200,
 'Analysis': {'AnalysisId': '5aeb6958-4e0c-4fe0-b720-a4d9750612f9',
  'Arn': 'arn:aws:quicksight:us-east-1:854807003520:analysis/5aeb6958-4e0c-4fe0-b720-a4d9750612f9',
  'Name': 'datadrift analysis',
  'Status': 'CREATION_SUCCESSFUL',
  'DataSetArns': ['arn:aws:quicksight:us-east-1:854807003520:dataset/54fb1fd7-9984-4400-a692-27e761375505'],
  'ThemeArn': 'arn:aws:quicksight::aws:theme/MIDNIGHT',
  'CreatedTime': datetime.datetime(2022, 5, 26, 14, 58, 52, 74000, tzinfo=tzlocal()),
  'LastUpdatedTime': datetime.datetime(2022, 5, 27, 3, 1, 27, 235000, tzinfo=tzlocal()),
  'Sheets': [{'SheetId': '46b59411-df57-4a53-9302-5244198377d2',


In [29]:
analysis_id = describe_analysis['Analysis']['AnalysisId']

arn = describe_analysis['Analysis']['Arn']

dataset_arn = describe_analysis['Analysis']['DataSetArns'][0]

# Creating the data Source

In [30]:
#________________________ Function to create data sources_______________________________________________________
def create_ds(client, account_id,Data_source_Id,data_source_name,DataSourceParameter,SslProperty,types):
    DS_creation = client.create_data_source(
    AwsAccountId=account_id,
    DataSourceId=Data_source_Id,      #Ds_id, #we'll provide the ds id
    Name=data_source_name, #display name for the data source
    Type=types,
    DataSourceParameters=DataSourceParameter,
#     Credentials=Credential,
#     Permissions=[
#         {
#             'Principal': arn,
#             'Actions': [ "athena:*", "quicksight:*", "quicksight:CreateDataSource"]
#         },
#     ],
#     VpcConnectionProperties={
#         'VpcConnectionArn': 'string'
#     },
    SslProperties=SslProperty)
    return DS_creation

In [31]:
ds_Name = "DataDrift_MLOPs"
ds_Id = '24e6bcd0-932e-4a9a-8b9d-bdf43bc09935'
# ds_Id = describe_data_source['DataSource']['DataSourceId']
# ds_Name = describe_data_source['DataSource']['Name']
ds_parameters = describe_data_source['DataSource']['DataSourceParameters']
ssl = describe_data_source['DataSource']['SslProperties']
type_ds = describe_data_source['DataSource']['Type']

In [32]:
create_ds(client, account_id, ds_Id, ds_Name, ds_parameters, ssl, type_ds)

{'ResponseMetadata': {'RequestId': '9a8193b0-b14b-47e1-b283-f3dd02fb9a29',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:39:51 GMT',
   'content-type': 'application/json',
   'content-length': '244',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9a8193b0-b14b-47e1-b283-f3dd02fb9a29'},
  'RetryAttempts': 0},
 'Status': 202,
 'Arn': 'arn:aws:quicksight:us-east-1:854807003520:datasource/24e6bcd0-932e-4a9a-8b9d-bdf43bc09935',
 'DataSourceId': '24e6bcd0-932e-4a9a-8b9d-bdf43bc09935',
 'CreationStatus': 'CREATION_IN_PROGRESS',
 'RequestId': '9a8193b0-b14b-47e1-b283-f3dd02fb9a29'}

# Creating Dataset

In [33]:

#_____________________________ Variables from the desribe dataset __________________________________________________
Import_Mode = describe_dataset['DataSet']['ImportMode']
Logical_TableMap=describe_dataset['DataSet']['LogicalTableMap']
Physical_TableMap=describe_dataset['DataSet']['PhysicalTableMap']
DataSet_Usage_Configuration = describe_dataset['DataSet']['DataSetUsageConfiguration']
dataset_Id = 'f1157d56-d6bb-48cc-8008-cc72d1sfc3cf'
dataset_Name = 'Drift_Dataset_MLOPs'

#creating dataset

def create_dataset(client, account_id, dataset_id,dataset_name,Physical_TableMap,Logical_TableMap,Import_Mode,DataSet_Usage_Configuration ):
    output = client.create_data_set(
    AwsAccountId=account_id,
    DataSetId=dataset_id,
    Name=dataset_name, #dataset_name,
    PhysicalTableMap=Physical_TableMap,
    LogicalTableMap=Logical_TableMap,
    ImportMode=Import_Mode,    
    DataSetUsageConfiguration=DataSet_Usage_Configuration )
    return output

In [34]:
create_dataset(client, 
               account_id, 
               dataset_Id,
               dataset_Name,
               Physical_TableMap,
               Logical_TableMap,
               Import_Mode,
               DataSet_Usage_Configuration )

{'ResponseMetadata': {'RequestId': 'a41235e6-096d-41ff-8295-1d160c5cf9a9',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:40:42 GMT',
   'content-type': 'application/json',
   'content-length': '237',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a41235e6-096d-41ff-8295-1d160c5cf9a9'},
  'RetryAttempts': 0},
 'Status': 201,
 'Arn': 'arn:aws:quicksight:us-east-1:854807003520:dataset/f1157d56-d6bb-48cc-8008-cc72d1sfc3cf',
 'DataSetId': 'f1157d56-d6bb-48cc-8008-cc72d1sfc3cf',
 'RequestId': 'a41235e6-096d-41ff-8295-1d160c5cf9a9'}

# Template Creation

In [35]:
# ------------------------ List template ------------------------------
list_template = client.list_templates(AwsAccountId=account_id)

list_template


{'ResponseMetadata': {'RequestId': '55e12be7-85a1-4620-85a5-d6c7c7b3bac1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:40:51 GMT',
   'content-type': 'application/json',
   'content-length': '365',
   'connection': 'keep-alive',
   'x-amzn-requestid': '55e12be7-85a1-4620-85a5-d6c7c7b3bac1'},
  'RetryAttempts': 0},
 'Status': 200,
 'TemplateSummaryList': [{'Arn': 'arn:aws:quicksight:us-east-1:854807003520:template/0696bc2d-b446-4de1-922c-1569b20ff97d',
   'TemplateId': '0696bc2d-b446-4de1-922c-1569b20ff97d',
   'Name': 'mlops_scoring_template',
   'LatestVersionNumber': 1,
   'CreatedTime': datetime.datetime(2022, 6, 2, 15, 31, 1, 193000, tzinfo=tzlocal()),
   'LastUpdatedTime': datetime.datetime(2022, 6, 2, 15, 31, 1, 193000, tzinfo=tzlocal())}],
 'RequestId': '55e12be7-85a1-4620-85a5-d6c7c7b3bac1'}

In [36]:
#-------------------------------- Creating template from analysis in DEV environment -------------------------

def creating_template_from_analysis(account_id, Template_Id, template_name, analysis_arn, dataset_placeholder, dataset_arn ):
    response = client.create_template(
    AwsAccountId=account_id,
    TemplateId=Template_Id,
    Name=template_name,
    SourceEntity={
        'SourceAnalysis': {
            'Arn': analysis_arn,
            'DataSetReferences': [
                {
                    'DataSetPlaceholder': dataset_placeholder,  #name for data
                    'DataSetArn': dataset_arn
                },
            ]
        } })
    return response

In [37]:
analysis_id = describe_analysis['Analysis']['AnalysisId']

arn = describe_analysis['Analysis']['Arn']

dataset_arn = describe_analysis['Analysis']['DataSetArns'][0]

Template_Id = '0696bc2d-b446-4de1-922c-1569b29ff97d'

template_name = 'mlops_data_drift_template'

dataset_placeholder = 'mlops_data_drift_placeholder'

In [38]:
creating_template_from_analysis(account_id, 
                                Template_Id, 
                                template_name, 
                                arn, 
                                dataset_placeholder, 
                                dataset_arn )

{'ResponseMetadata': {'RequestId': 'c62e6e6e-72ae-435a-9014-0d9c6339023e',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 03 Jun 2022 01:41:33 GMT',
   'content-type': 'application/json',
   'content-length': '353',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c62e6e6e-72ae-435a-9014-0d9c6339023e'},
  'RetryAttempts': 0},
 'Status': 202,
 'Arn': 'arn:aws:quicksight:us-east-1:854807003520:template/0696bc2d-b446-4de1-922c-1569b29ff97d',
 'VersionArn': 'arn:aws:quicksight:us-east-1:854807003520:template/0696bc2d-b446-4de1-922c-1569b29ff97d/version/1',
 'TemplateId': '0696bc2d-b446-4de1-922c-1569b29ff97d',
 'CreationStatus': 'CREATION_IN_PROGRESS',
 'RequestId': 'c62e6e6e-72ae-435a-9014-0d9c6339023e'}

In [ ]:
#________________________ Function to create data sources_______________________________________________________
def create_ds(client, account_id,Data_source_Id,data_source_name,DataSourceParameter,SslProperty):
    DS_creation = client.create_data_source(
    AwsAccountId=account_id,
    DataSourceId=Data_source_Id,      #Ds_id, #we'll provide the ds id
    Name=data_source_name, #display name for the data source
    Type='ATHENA',
    DataSourceParameters=DataSourceParameter,
#     Credentials=Credential,
#     Permissions=[
#         {
#             'Principal': arn,
#             'Actions': [ "athena:*", "quicksight:*", "quicksight:CreateDataSource"]
#         },
#     ],
#     VpcConnectionProperties={
#         'VpcConnectionArn': 'string'
#     },
    SslProperties=SslProperty)
    return DS_creation

# _____________________________________List dataset _______________________________________________________________
list_dataset = quicksight.list_data_sets(AwsAccountId=account_id)


# _________________________________Describe dataset ________________________________________________________________
describe_dataset = client.describe_data_set(AwsAccountId=account_id, DataSetId= '05dbeb9e-6ce3-475e-bc58-6664e87992be')

#_____________________________ Variables from the desribe dataset __________________________________________________
Import_Mode = describe_dataset['DataSet']['ImportMode']
Logical_TableMap=describe_dataset['DataSet']['LogicalTableMap']
Physical_TableMap=describe_dataset['DataSet']['PhysicalTableMap']
DataSet_Usage_Configuration = describe_dataset['DataSet']['DataSetUsageConfiguration']
dataset_Id = describe_dataset['DataSet']['DataSetId']
dataset_Name = describe_dataset['DataSet']['Name']

#creating dataset

def create_dataset(client, account_id, dataset_id,dataset_name,Physical_TableMap,Logical_TableMap,Import_Mode,DataSet_Usage_Configuration ):
    output = client.create_data_set(
    AwsAccountId=account_id,
    DataSetId=dataset_id,
    Name=dataset_name, #dataset_name,
    PhysicalTableMap=Physical_TableMap,
    LogicalTableMap=Logical_TableMap,
    ImportMode=Import_Mode,    
    DataSetUsageConfiguration=DataSet_Usage_Configuration )
    return output


# ------------------------ List template ------------------------------
list_template = client.list_templates(AwsAccountId=account_id)

des_template = client.list_templates(AwsAccountId=account_id,TemplateId=templateID)

template_arn = des_template['Template']['Arn']


#-------------------------------- Creating template from analysis in DEV environment -------------------------

def creating_template_from_analysis(account_id, Template_Id, template_name, analysis_arn, dataset_placeholder, dataset_arn ):
    response = client.create_template(
    AwsAccountId=account_id,
    TemplateId=Template_Id,
    Name=template_name,
    SourceEntity={
        'SourceAnalysis': {
            'Arn': analysis_arn,
            'DataSetReferences': [
                {
                    'DataSetPlaceholder': dataset_placeholder,  #name for data
                    'DataSetArn': dataset_arn
                },
            ]
        } })
    return response

# -------------------------Creating template from template -------------------------------------------
def create_template_from_template(account_id,Template_Id,template_name,tempalte_arn):
    response = client.create_template(
    AwsAccountId=account_id,
    TemplateId=Template_Id,
    Name=template_name,
    SourceEntity={
        'SourceTemplate': {
            'Arn': tempalte_arn
        }
    })
    return response



# ---------------------------- List analysis ---------------------------------
list_analysis = client.list_analyses(AwsAccountId=account_id)

### --------------------- Desribe analysis -------------------------------------------------------
describe_analysis = client.describe_analysis(AwsAccountId=account_id, AnalysisId=analysis_id)

# ----------------------- variables from analysis description ----------------------------------------
analysis_id = describe_analysis['Analysis']['AnalysisId']
arn = describe_analysis['Analysis']['Arn']
dataset_arn = describe_analysis['Analysis']['DataSetArns'][0]


# --------------------   Creating analysis from template -----------------------------------------
def creating_analysis(client, account_id, analysis_id, analysis_name,dataset_placeholder,dataset_arn, template_arn):
    response = client.create_analysis(
    AwsAccountId=account_id,
    AnalysisId=analysis_id,
    Name=analysis_name,
    SourceEntity={
        'SourceTemplate': {
            'DataSetReferences': [
                {
                    'DataSetPlaceholder': dataset_placeholder,
                    'DataSetArn': dataset_arn
                },
            ],
            "Arn":template_arn,
                }
            }
        )
    return response


def creating_dashboard(client,account_id,dashboard_id,dashboard_name,Iam_arn,dataset_placeholder,dataset_arn,template_arn,):
    response = client.create_dashboard(
    AwsAccountId=account_id,
    DashboardId=dashboard_id, #The ID for the dashboard, also added to the IAM policy.
    Name=dashboard_name,
    Permissions=[
        {
            'Principal': Iam_arn,
            'Actions': [
                'quicksight:DescribeDashboard',
                'quicksight:ListDashboardVersions',
                'quicksight:UpdateDashboardPermissions',
                'quicksight:QueryDashboard',
                'quicksight:UpdateDashboard',
                'quicksight:DeleteDashboard',
                'quicksight:DescribeDashboardPermissions',
                'quicksight:UpdateDashboardPublishedVersion'
            ]
        },
    ],
    SourceEntity={
        'SourceTemplate': {
            'DataSetReferences': [
                {
                    'DataSetPlaceholder': dataset_placeholder,
                    'DataSetArn': dataset_arn
                },
            ],
            'Arn': template_arn
        }
    },
    VersionDescription='1.0',
    DashboardPublishOptions={
        'AdHocFilteringOption': {
            'AvailabilityStatus': 'ENABLED'|'DISABLED'
        },
        'ExportToCSVOption': {
            'AvailabilityStatus': 'ENABLED'|'DISABLED'
        },
        'SheetControlsOption': {
            'VisibilityState': 'EXPANDED'|'COLLAPSED'
        }
    })
    return response



    def updating_template(client, account_id,Template_id, template_arn,template_version, template_name):
        response = client.update_template(
        AwsAccountId=account_id,
        TemplateId=Template_id,
        SourceEntity={
            'SourceTemplate': {
                'Arn': template_arn
            }
        },
        VersionDescription=template_version,
        Name=template_name)
        return response